In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import random

In [8]:
with open('/content/drive/MyDrive/NLP_GLAssignment/GL Bot.json') as data_file:
  data = json.load(data_file)

In [10]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)


In [12]:
labels

['Intro', 'Exit', 'Olympus', 'SL', 'NN', 'Bot', 'Profane', 'Ticket']

### Labelencoding and pickling the encoder

In [19]:
encoder = LabelEncoder()
encoder.fit(training_labels)
with open(r"/content/drive/MyDrive/NLP_GLAssignment/chatbotLablelEncoder.pkl",'wb') as f:
  pickle.dump(encoder,f)

In [20]:
encoded_labels = encoder.transform(training_labels)
encoded_labels

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7])

### Text Preprocessing

By using tokenizer we can not only remove unwanted out of vocabulary words(using oov token), we can also remove unwanted punctuation marks along with vectorization of texts

In [21]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [37]:
with open(r'/content/drive/MyDrive/NLP_GLAssignment/chatbot_tokenizer.pkl','wb') as f:
  pickle.dump(tokenizer,f)

### Training the deep neural network to predict the tags based on user inputs

In [38]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d_4 ( (None, 16)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_13 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_14 (Dense)             (None, 8)                 136       
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
_________________________________________________________________


In [34]:
epochs = 200
history = model.fit(padded_sequences, np.array(encoded_labels), epochs=epochs)

Epoch 1/200
4/4 [==============================] - 1s 3ms/step - loss: 2.0800 - accuracy: 0.0781
Epoch 2/200
4/4 [==============================] - 0s 4ms/step - loss: 2.0766 - accuracy: 0.2000
Epoch 3/200
4/4 [==============================] - 0s 4ms/step - loss: 2.0727 - accuracy: 0.2437
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 2.0700 - accuracy: 0.2281
Epoch 5/200
4/4 [==============================] - 0s 4ms/step - loss: 2.0677 - accuracy: 0.2406
Epoch 6/200
4/4 [==============================] - 0s 5ms/step - loss: 2.0623 - accuracy: 0.2406
Epoch 7/200
4/4 [==============================] - 0s 9ms/step - loss: 2.0608 - accuracy: 0.2531
Epoch 8/200
4/4 [==============================] - 0s 5ms/step - loss: 2.0553 - accuracy: 0.2198
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 2.0553 - accuracy: 0.2052
Epoch 10/200
4/4 [==============================] - 0s 6ms/step - loss: 2.0455 - accuracy: 0.2281
Epoch 11/200
4/4 [===========

In [36]:
model.save(r"/content/drive/MyDrive/NLP_GLAssignment/chatbot_model.h5",history)